In [1]:
# pip install selenium
import os

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

PATH = os.path.join(os.getcwd(), "chromedriver.exe")

# Create a Chrome options object with the headless option
options = Options()
options.headless = True

search_keys = ['眼睛紅','呼吸道感染', '糖尿病','耳鳴','落髮','憂鬱'] 

C:\Users\julia\AppData\Local\Temp/ipykernel_36648/1864071614.py:12: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [3]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time

In [4]:
%%time
links = []
titles = []
dates = []
contents = []
categories = []
item_id = []
photo_links = []

for key in search_keys:
    # Create a Chrome driver with the options object
    driver = webdriver.Chrome(PATH, options=options)

    driver.get("https://heho.com.tw/symptom-query")
    
    element = driver.find_element(By.ID, "name")
    element.send_keys(key)

    current_window = driver.current_window_handle
    driver.find_element(By.ID, "input1").click()
    # switch to the new window
    for window_handle in driver.window_handles:
        if window_handle != current_window:
            driver.switch_to.window(window_handle)
            break
    
    # get the new page URL
    new_page_url = driver.current_url
    # search_page_url.append(new_page_url)
    # print(new_page_url)
    print("Getting categorical articles:", key)
    # driver.quit()

    driver.get(new_page_url)
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "gsc-wrapper")))

    # print(f"Element of page {i+1}: {element.text}")
    content = element.get_attribute("innerHTML")
    page = BeautifulSoup(content, 'lxml')
    # page

    # 抓新聞列表
    items = page.find('div', {'class': "gsc-expansionArea"}).findAll('div', {'class': "gs-webResult gs-result"})
    # print(items)

    serial_no = 1
    for j, item in enumerate(items): #針對每一篇項目 抓其細節
        
        # title
        title = item.find('div', {"class":'gs-title'}).text
        print(serial_no,'--', title )
        # print(title)


        # news link
        link = item.find('a').get('href')
        # print(link)


        # photo link
        try:
            if item.find('img').has_attr('src'):
                photo_link = item.find('img').get('src')
            else:
                photo_link = item.find('img').get('src')
            print(photo_link)
        except:
            photo_link = ''


        # Request the categorical news page
        inner_req = requests.get(link)
        inner_page = BeautifulSoup(inner_req.text, 'lxml')
        inner_items = inner_page.find('div', {'class': re.compile("article-inner")})
        

        # date 
        news_date = inner_items.find('time', {'class':"entry-date published"}).text
        dtime = datetime.strptime(news_date,  "%Y 年 %m 月 %d 日")
        news_date_str = dtime.strftime("%Y-%m-%d")
        dtstr = dtime.strftime("%Y%m%d")
        # print(news_date_str)


        # content
        content = inner_items.find('div',{'class':"entry-content single-page"})


        # 額外插入的延伸閱讀區塊-->移除之
        try:
            content.find('ul').decompose()
        except:
            pass

        cont = content.text
        cont = re.sub('^延伸閱讀.*$', '', cont, flags=re.DOTALL | re.MULTILINE)
        cont = re.sub(r'.*(文|/|圖).*', '', cont)

        
        item_id.append(key + "_" + dtstr + "_" + str(serial_no)) # format of serial number: aipl_20220314_1
        categories.append(key)
        contents.append(cont)
        titles.append(title)
        links.append(link)
        photo_links.append(photo_link)
        dates.append(news_date_str)

        serial_no += 1

        time.sleep(2)

        if (j==4): # Crawl only 5 pieces of news for each category.
            break 
    

<timed exec>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


Getting categorical articles: 眼睛紅
1 -- 眼睛紅卻不痛不癢？你可能是「結膜下出血」，原因、解方一次看 ...
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSZRQPqWGb5mSieKzMTrjKScxeunV_AB6qeWA0affA_pRQ9nPG3oM9U8GQ
2 -- 「紅眼睛」是角膜炎、結膜炎？一圖看懂兩種眼疾差異、治療方法 ...
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTm0GdwdaoePnVFXbH0bNhddRkeo-ykXBa8qhFBKa91CfRf89l3IAJkq_U
3 -- 常常眼睛紅紅、帶血絲？比起點眼藥水這2招更有效！ - Heho健康
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQMVmXqdTiQ9CUvVFEwDquGUw4z4YEhslw6nA7N4rB1dmIwodIe7a5AiKQ
4 -- 早上起床眼睛紅紅，是乾眼症還是結膜炎？ - Heho健康
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTeUX2s_en7KwKHZ67Ofw3yXmXebtaFvYV7-TiUny1H4SxBswPuq-GRYoM
5 -- 洗完澡眼睛紅通通除了水跑進眼睛還有這個原因- Heho健康
https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQ9fe3VUFrIGtEohv5XBHt7-uRo4lS2B8Dd0dGIkmCunpwC0wHYn5u72Ew
Getting categorical articles: 呼吸道感染
1 -- 緩解6 年咳嗽病史！秋冬上呼吸道感染症狀多，中醫治療這樣看- Heho ...
https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQXLMj4AwV_90Etm1-E43FhahuL5isxvbhd00I2orWBjksfwVOjJtr6cWI
2 -- 小小孩的呼吸道殺手！認識「呼吸道融合病毒」 

In [5]:
contents = [re.sub('\n', '', cont) for cont in contents]
contents

['小露一早起床照鏡子時發現自己的眼睛紅紅的，眼白的地方像是在流血一樣，但不痛也不癢，除了紅紅的看起來很恐怖，也沒有其他不舒服的感覺，眼科醫師說這是結膜下出血，主要是因為太過勞累，請小露要多休息。結膜下出血是什麼？簡單來說，結膜下出血是眼睛的微血管破裂，血流到眼球和結膜之間，眼白的地方像是被血染紅了一樣。結膜下出血除了眼睛紅紅的之外，幾乎不會有其他像是腫、痛、癢不舒服的感覺，也不會影響視力，看東西一樣可以很清楚。造成結膜下出血的原因：過度用力：劇烈咳嗽、用力解便、搬重物、有外力衝擊導致微血管破裂。揉眼睛：揉眼會壓迫眼球，當眼球被擠壓血管就容易出血。氣溫變化太大：忽冷忽熱，眼睛的血管無法正常收縮也可能會發生血管破裂的情形。而若有合併眼球外傷，就要做比較詳細的檢查，看看是否為鞏膜裂傷或是眼球鈍挫傷，而非單純的結膜下出血。另外，如果一直反覆出現結膜下出血，則很可能跟全身性的疾病有關，像是高血壓、糖尿病會讓血管的彈性變差、構造變脆弱，血管也因此一受到刺激就容易破裂。結膜下出血要怎麼預防？結膜下出血通常會在 7 到 12 天內自行吸收，眼白上的血塊會慢慢散開，由血紅色轉為黃棕色到完全消失，通常不需要做特別的治療，就像瘀青也會自己慢慢好起來一樣。要預防結膜下出血，就要保護好眼睛的血管。避免讓身體的各個部位過度用力：用力憋氣、用力咳嗽、嘔吐、用力解便都算是過度用力。若有便秘的情形，可以多吃蔬菜水果、多補充水分、多運動，必要時可以請醫師開立軟便劑。避免揉眼：有意識的注意到自己是否有用手揉眼睛，養成不要用手摸眼睛的習慣。避免讓眼睛直吹冷風：氣溫突然變低，血管突然收縮，血壓急速上升就可能使血管突然爆開、出血，可以戴上墨鏡或是天冷風大就盡量不要出門。控制血壓、血糖：平時做好體重控制、規律運動、多吃粗食和蔬菜，以少油、少鹽、少糖、高纖維的食材為主。單純的結膜下出血並非嚴重的疾病，多休息、不要讓眼睛過勞，讓眼睛的血管保持在一個良好的狀態，就不會輕易發生結膜下出血的情況。參考資料：高醫醫訊月刊',
 '眼睛好癢伴隨著刺痛感，沒想到越去揉眼睛，反而越紅腫，進而出現嚇人的紅眼，大部分的人第一直覺都覺得是「結膜炎」釀禍，但其實也有可能是「角膜炎」，兩者都有可能出現紅眼睛，但兩者疾病不僅治療方法不同，連嚴重程度也大不相同。結膜炎、角膜炎預後大不同正看眼睛黑白分明，包含了中間黑色的角膜，與旁邊白色的結膜

### **PANDA資料處理**

In [6]:
import pandas as pd

data = zip(item_id, dates, categories, titles, contents, links, photo_links)
df = pd.DataFrame(list(data), columns=['item_id','date','category','title','content','link','photo_link'])

In [7]:
df.head(5)

,item_id,date,category,title,content,link,photo_link
0,眼睛紅_20210221_1,2021-02-21,眼睛紅,眼睛紅卻不痛不癢？你可能是「結膜下出血」，原因、解方一次看 ...,小露一早起床照鏡子時發現自己的眼睛紅紅的，眼白的地方像是在流血一樣，但不痛也不癢，除了紅紅的...,https://heho.com.tw/archives/157395,https://encrypted-tbn1.gstatic.com/images?q=tb...
1,眼睛紅_20210218_2,2021-02-18,眼睛紅,「紅眼睛」是角膜炎、結膜炎？一圖看懂兩種眼疾差異、治療方法 ...,眼睛好癢伴隨著刺痛感，沒想到越去揉眼睛，反而越紅腫，進而出現嚇人的紅眼，大部分的人第一直覺都...,https://heho.com.tw/archives/81551,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,眼睛紅_20190821_3,2019-08-21,眼睛紅,常常眼睛紅紅、帶血絲？比起點眼藥水這2招更有效！ - Heho健康,雖然大家會說眼睛是我們的「靈魂之窗」，但卻很少人會每天盯著自己的眼睛觀察，總是要別人大叫「哇...,https://heho.com.tw/archives/55186,https://encrypted-tbn1.gstatic.com/images?q=tb...
3,眼睛紅_20181128_4,2018-11-28,眼睛紅,早上起床眼睛紅紅，是乾眼症還是結膜炎？ - Heho健康,小威最近眼睛老是紅紅的，他覺得是乾眼問題，就去買了眼藥水回來點，結果隔了好幾天都沒有恢復，而...,https://heho.com.tw/archives/28892,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,眼睛紅_20190525_5,2019-05-25,眼睛紅,洗完澡眼睛紅通通除了水跑進眼睛還有這個原因- Heho健康,大部分人都有過這樣的經驗，洗完澡後全身清爽，一照鏡子卻發現自己竟然像兔子一樣紅了眼，讓你忍不...,https://heho.com.tw/archives/49906,https://encrypted-tbn1.gstatic.com/images?q=tb...


In [8]:
df.to_csv("heho_categories_articles.csv", sep="|", index=False)

In [9]:
df = pd.read_csv("heho_categories_articles.csv", sep="|")
# df.head(2)
print(df.content[0:5])
print(df.shape)

0    小露一早起床照鏡子時發現自己的眼睛紅紅的，眼白的地方像是在流血一樣，但不痛也不癢，除了紅紅的...
1    眼睛好癢伴隨著刺痛感，沒想到越去揉眼睛，反而越紅腫，進而出現嚇人的紅眼，大部分的人第一直覺都...
2    雖然大家會說眼睛是我們的「靈魂之窗」，但卻很少人會每天盯著自己的眼睛觀察，總是要別人大叫「哇...
3    小威最近眼睛老是紅紅的，他覺得是乾眼問題，就去買了眼藥水回來點，結果隔了好幾天都沒有恢復，而...
4    大部分人都有過這樣的經驗，洗完澡後全身清爽，一照鏡子卻發現自己竟然像兔子一樣紅了眼，讓你忍不...
Name: content, dtype: object
(30, 7)


## _**統計各網頁關鍵字**_

In [10]:
import numpy

In [11]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")


### **WS 斷詞**

In [12]:
%%time
# It takes time.
tokens=ws(df.content)

Inference: 100%|██████████| 1/1 [00:22<00:00, 22.77s/it]

Wall time: 23.2 s


### **POS詞性標記**

In [13]:
%%time
tokens_pos = pos(tokens)

Inference: 100%|██████████| 10/10 [01:30<00:00,  9.03s/it]

Wall time: 1min 30s


In [14]:
print("1.",len(tokens_pos))
print("2.",len(tokens_pos[2]))
print("3.",tokens_pos[0])

1. 30
2. 646
3. ['VC', 'D', 'VA', 'VC', 'Na', 'Ng', 'VE', 'Nh', 'DE', 'Na', 'VH', 'T', 'COMMACATEGORY', 'VH', 'DE', 'Na', 'P', 'SHI', 'P', 'VA', 'VH', 'COMMACATEGORY', 'Cbb', 'D', 'VH', 'D', 'D', 'VH', 'COMMACATEGORY', 'P', 'VH', 'DE', 'D', 'Dfa', 'VH', 'COMMACATEGORY', 'D', 'VJ', 'Neqa', 'D', 'VH', 'DE', 'Na', 'COMMACATEGORY', 'Nc', 'Na', 'VE', 'Nep', 'SHI', 'Na', 'Ng', 'VH', 'COMMACATEGORY', 'D', 'SHI', 'Cbb', 'Dfa', 'Dfa', 'VH', 'COMMACATEGORY', 'VF', 'VH', 'D', 'D', 'VA', 'PERIODCATEGORY', 'Na', 'Ng', 'VH', 'SHI', 'Nep', 'QUESTIONCATEGORY', 'VH', 'D', 'VE', 'COMMACATEGORY', 'Na', 'Ng', 'VH', 'SHI', 'Na', 'DE', 'Na', 'VH', 'COMMACATEGORY', 'Na', 'VCL', 'Na', 'Caa', 'Na', 'Ng', 'COMMACATEGORY', 'VH', 'VH', 'DE', 'Na', 'SHI', 'P', 'Na', 'VC', 'Di', 'VH', 'PERIODCATEGORY', 'Na', 'Ng', 'VH', 'P', 'Na', 'VH', 'DE', 'Ng', 'COMMACATEGORY', 'Da', 'D', 'V_2', 'Neqa', 'SHI', 'VH', 'PAUSECATEGORY', 'VH', 'PAUSECATEGORY', 'VH', 'D', 'VH', 'DE', 'Na', 'COMMACATEGORY', 'D', 'D', 'VC', 'Na', 'COMM

### **NER 實體辨識**

In [15]:
%%time
entity_list = ner(df.content)

Inference: 100%|██████████| 1/1 [00:19<00:00, 19.91s/it]


Wall time: 20.7 s


In [16]:
entity_list[1]

[NerToken(word='第一', ner='ORDINAL', idx=(41, 43)),
 NerToken(word='兩', ner='CARDINAL', idx=(71, 72)),
 NerToken(word='陳瑩山', ner='PERSON', idx=(179, 182)),
 NerToken(word='兩', ner='CARDINAL', idx=(259, 260)),
 NerToken(word='陳瑩山', ner='PERSON', idx=(267, 270)),
 NerToken(word='第一', ner='ORDINAL', idx=(391, 393)),
 NerToken(word='陳瑩山', ner='PERSON', idx=(436, 439)),
 NerToken(word='陳瑩山', ner='PERSON', idx=(529, 532)),
 NerToken(word='5', ner='CARDINAL', idx=(564, 565)),
 NerToken(word='10天', ner='DATE', idx=(566, 569)),
 NerToken(word='陳瑩山', ner='PERSON', idx=(647, 650)),
 NerToken(word='陳瑩山', ner='PERSON', idx=(822, 825))]

### **關鍵字與詞性搭配 (ws & pos)**

In [17]:
word_pos_pair = [list(zip(w,p)) for w, p in zip(tokens, tokens_pos)]

print("1.", len(word_pos_pair))
print("2.", word_pos_pair[1][0])

1. 30
2. ('眼睛', 'Na')


In [18]:
word_pos_pair

[[('小露', 'VC'),
  ('一早', 'D'),
  ('起床', 'VA'),
  ('照', 'VC'),
  ('鏡子', 'Na'),
  ('時', 'Ng'),
  ('發現', 'VE'),
  ('自己', 'Nh'),
  ('的', 'DE'),
  ('眼睛', 'Na'),
  ('紅紅', 'VH'),
  ('的', 'T'),
  ('，', 'COMMACATEGORY'),
  ('眼白', 'VH'),
  ('的', 'DE'),
  ('地方', 'Na'),
  ('像', 'P'),
  ('是', 'SHI'),
  ('在', 'P'),
  ('流血', 'VA'),
  ('一樣', 'VH'),
  ('，', 'COMMACATEGORY'),
  ('但', 'Cbb'),
  ('不', 'D'),
  ('痛', 'VH'),
  ('也', 'D'),
  ('不', 'D'),
  ('癢', 'VH'),
  ('，', 'COMMACATEGORY'),
  ('除了', 'P'),
  ('紅紅', 'VH'),
  ('的', 'DE'),
  ('看起來', 'D'),
  ('很', 'Dfa'),
  ('恐怖', 'VH'),
  ('，', 'COMMACATEGORY'),
  ('也', 'D'),
  ('沒有', 'VJ'),
  ('其他', 'Neqa'),
  ('不', 'D'),
  ('舒服', 'VH'),
  ('的', 'DE'),
  ('感覺', 'Na'),
  ('，', 'COMMACATEGORY'),
  ('眼科', 'Nc'),
  ('醫師', 'Na'),
  ('說', 'VE'),
  ('這', 'Nep'),
  ('是', 'SHI'),
  ('結膜', 'Na'),
  ('下', 'Ng'),
  ('出血', 'VH'),
  ('，', 'COMMACATEGORY'),
  ('主要', 'D'),
  ('是', 'SHI'),
  ('因為', 'Cbb'),
  ('太', 'Dfa'),
  ('過', 'Dfa'),
  ('勞累', 'VH'),
  ('，', 'COMMACATEGORY

### **過濾**

In [19]:
allowPOS=['Na','VA','VC']

tokens_v2 =[]
for wp in word_pos_pair:
    tokens_v2.append([w for w,p in wp if (len(w) >= 2) and p in allowPOS])

tokens_v2[0]

['小露',
 '起床',
 '鏡子',
 '眼睛',
 '地方',
 '流血',
 '感覺',
 '醫師',
 '結膜',
 '休息',
 '結膜',
 '結膜',
 '眼睛',
 '微血管',
 '眼球',
 '結膜',
 '地方',
 '染紅',
 '結膜',
 '眼睛',
 '感覺',
 '影響',
 '視力',
 '東西',
 '結膜',
 '下出血',
 '原因',
 '咳嗽',
 '解便',
 '外力',
 '衝擊',
 '微血管',
 '眼睛',
 '壓迫',
 '眼球',
 '眼球',
 '擠壓',
 '血管',
 '氣溫',
 '變化',
 '眼睛',
 '血管',
 '血管',
 '情形',
 '合併',
 '眼球',
 '外傷',
 '鞏膜',
 '眼球',
 '挫傷，',
 '結膜下出血',
 '結膜下出血',
 '全身性',
 '疾病',
 '高血',
 '糖尿病',
 '血管',
 '彈性',
 '構造變',
 '血管',
 '刺激',
 '結膜下出血',
 '結膜下',
 '眼白',
 '血塊',
 '紅色',
 '黃棕色',
 '消失',
 '治療',
 '結膜',
 '下出血',
 '保護',
 '眼睛',
 '血管',
 '身體',
 '部位',
 '憋氣',
 '咳嗽',
 '嘔吐',
 '用力',
 '情形',
 '蔬菜',
 '水果',
 '補充',
 '水分',
 '運動',
 '醫師',
 '開立',
 '軟便劑',
 '眼睛',
 '養成',
 '眼睛',
 '習慣',
 '眼睛',
 '吹冷',
 '氣溫',
 '血管',
 '血壓',
 '上升',
 '血管',
 '墨鏡',
 '出門',
 '控制',
 '血壓',
 '血糖',
 '體重',
 '控制',
 '規律',
 '運動',
 '蔬菜',
 '纖維',
 '食材',
 '結膜',
 '疾病',
 '休息',
 '眼睛',
 '眼睛',
 '血管',
 '狀態',
 '結膜下',
 '情況',
 '參考',
 '資料',
 '月刊']

### **新增段詞資料欄位**

In [20]:
df['tokens'] = tokens
df['tokens_v2'] = tokens_v2
df['entities'] = entity_list
df['token_pos'] = word_pos_pair

In [21]:
# df 或 df.head()

### **統計字頻**

In [22]:
# allowPOS過濾條件:兩個字以上與特定的詞性
allowPOS=['Na','VA']

In [23]:
from collections import Counter
def word_frequency(word_pos_pair):
    filtered_words =[]
    for word, pos in word_pos_pair:
        if (pos in allowPOS) & (len(word) >= 2):            
            filtered_words.append(word)
        #print('%s %s' % (word, flag))
    c = Counter( filtered_words)
    return c.most_common(30)

# 看過濾後重新搭配的: word_pos_pair["第幾篇"]

In [24]:
words = word_frequency(word_pos_pair[0])
words

[('眼睛', 11),
 ('血管', 9),
 ('結膜', 8),
 ('眼球', 5),
 ('結膜下出血', 3),
 ('地方', 2),
 ('感覺', 2),
 ('醫師', 2),
 ('休息', 2),
 ('微血管', 2),
 ('咳嗽', 2),
 ('氣溫', 2),
 ('情形', 2),
 ('疾病', 2),
 ('結膜下', 2),
 ('蔬菜', 2),
 ('運動', 2),
 ('血壓', 2),
 ('起床', 1),
 ('鏡子', 1),
 ('流血', 1),
 ('視力', 1),
 ('東西', 1),
 ('原因', 1),
 ('外力', 1),
 ('衝擊', 1),
 ('變化', 1),
 ('外傷', 1),
 ('鞏膜', 1),
 ('全身性', 1)]

In [25]:
%%time
keyfreqs =[]
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)  

# keyfreqs[0]: 看第0篇過濾後斷詞的字頻
# keyfreqs[0:2]: 看第0篇與第1篇過濾後斷詞的字頻

# 再次新增欄位到資料集
df['top_key_freq'] = keyfreqs
df.head(3)

Wall time: 14.2 ms


,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq
0,眼睛紅_20210221_1,2021-02-21,眼睛紅,眼睛紅卻不痛不癢？你可能是「結膜下出血」，原因、解方一次看 ...,小露一早起床照鏡子時發現自己的眼睛紅紅的，眼白的地方像是在流血一樣，但不痛也不癢，除了紅紅的...,https://heho.com.tw/archives/157395,https://encrypted-tbn1.gstatic.com/images?q=tb...,"[小露, 一早, 起床, 照, 鏡子, 時, 發現, 自己, 的, 眼睛, 紅紅, 的, ，...","[小露, 起床, 鏡子, 眼睛, 地方, 流血, 感覺, 醫師, 結膜, 休息, 結膜, 結...","[( 7, FAC, (453, 455))]","[(小露, VC), (一早, D), (起床, VA), (照, VC), (鏡子, Na...","[(眼睛, 11), (血管, 9), (結膜, 8), (眼球, 5), (結膜下出血, ..."
1,眼睛紅_20210218_2,2021-02-18,眼睛紅,「紅眼睛」是角膜炎、結膜炎？一圖看懂兩種眼疾差異、治療方法 ...,眼睛好癢伴隨著刺痛感，沒想到越去揉眼睛，反而越紅腫，進而出現嚇人的紅眼，大部分的人第一直覺都...,https://heho.com.tw/archives/81551,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[眼睛, 好, 癢, 伴隨, 著, 刺痛感, ，, 沒想到, 越, 去, 揉, 眼睛, ，,...","[眼睛, 伴隨, 刺痛感, 眼睛, 紅眼, 直覺, 結膜炎, 角膜炎, 眼睛, 疾病, 治療...","[(第一, ORDINAL, (41, 43)), (兩, CARDINAL, (71, 7...","[(眼睛, Na), (好, Dfa), (癢, VH), (伴隨, VC), (著, Di...","[(角膜炎, 12), (眼睛, 11), (結膜炎, 10), (角膜, 10), (結膜..."
2,眼睛紅_20190821_3,2019-08-21,眼睛紅,常常眼睛紅紅、帶血絲？比起點眼藥水這2招更有效！ - Heho健康,雖然大家會說眼睛是我們的「靈魂之窗」，但卻很少人會每天盯著自己的眼睛觀察，總是要別人大叫「哇...,https://heho.com.tw/archives/55186,https://encrypted-tbn1.gstatic.com/images?q=tb...,"[雖然, 大家, 會, 說, 眼睛, 是, 我們, 的, 「, 靈魂, 之, 窗, 」, ，...","[眼睛, 靈魂, 眼睛, 眼睛, 白兔, 眼睛, 症狀, 動力, 醫生, 眼藥, 眼睛, 眼...","[(0分, TIME, (787, 789)), (10分, TIME, (809, 812))]","[(雖然, Cbb), (大家, Nh), (會, D), (說, VE), (眼睛, Na...","[(眼睛, 30), (眼藥水, 5), (問題, 3), (血管, 3), (時候, 3)..."


### **資料表順序整理**

In [26]:
df.columns

Index(['item_id', 'date', 'category', 'title', 'content', 'link', 'photo_link',
       'tokens', 'tokens_v2', 'entities', 'token_pos', 'top_key_freq'],
      dtype='object')

In [27]:
df=df[['item_id', 'date', 'category', 'title', 'content', 'top_key_freq', 'tokens', 'tokens_v2', 'entities', 'token_pos', 'link', 'photo_link']]
df.head(1)

,item_id,date,category,title,content,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,眼睛紅_20210221_1,2021-02-21,眼睛紅,眼睛紅卻不痛不癢？你可能是「結膜下出血」，原因、解方一次看 ...,小露一早起床照鏡子時發現自己的眼睛紅紅的，眼白的地方像是在流血一樣，但不痛也不癢，除了紅紅的...,"[(眼睛, 11), (血管, 9), (結膜, 8), (眼球, 5), (結膜下出血, ...","[小露, 一早, 起床, 照, 鏡子, 時, 發現, 自己, 的, 眼睛, 紅紅, 的, ，...","[小露, 起床, 鏡子, 眼睛, 地方, 流血, 感覺, 醫師, 結膜, 休息, 結膜, 結...","[( 7, FAC, (453, 455))]","[(小露, VC), (一早, D), (起床, VA), (照, VC), (鏡子, Na...",https://heho.com.tw/archives/157395,https://encrypted-tbn1.gstatic.com/images?q=tb...


In [28]:
df.to_csv('heho_dataset_processed.csv', sep='|', index=False)

In [29]:
df = pd.read_csv('heho_dataset_processed.csv', sep='|')
df.head(3)

,item_id,date,category,title,content,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,眼睛紅_20210221_1,2021-02-21,眼睛紅,眼睛紅卻不痛不癢？你可能是「結膜下出血」，原因、解方一次看 ...,小露一早起床照鏡子時發現自己的眼睛紅紅的，眼白的地方像是在流血一樣，但不痛也不癢，除了紅紅的...,"[('眼睛', 11), ('血管', 9), ('結膜', 8), ('眼球', 5), ...","['小露', '一早', '起床', '照', '鏡子', '時', '發現', '自己',...","['小露', '起床', '鏡子', '眼睛', '地方', '流血', '感覺', '醫師...","[NerToken(word=' 7', ner='FAC', idx=(453, 455))]","[('小露', 'VC'), ('一早', 'D'), ('起床', 'VA'), ('照'...",https://heho.com.tw/archives/157395,https://encrypted-tbn1.gstatic.com/images?q=tb...
1,眼睛紅_20210218_2,2021-02-18,眼睛紅,「紅眼睛」是角膜炎、結膜炎？一圖看懂兩種眼疾差異、治療方法 ...,眼睛好癢伴隨著刺痛感，沒想到越去揉眼睛，反而越紅腫，進而出現嚇人的紅眼，大部分的人第一直覺都...,"[('角膜炎', 12), ('眼睛', 11), ('結膜炎', 10), ('角膜', ...","['眼睛', '好', '癢', '伴隨', '著', '刺痛感', '，', '沒想到',...","['眼睛', '伴隨', '刺痛感', '眼睛', '紅眼', '直覺', '結膜炎', '...","[NerToken(word='第一', ner='ORDINAL', idx=(41, 4...","[('眼睛', 'Na'), ('好', 'Dfa'), ('癢', 'VH'), ('伴隨...",https://heho.com.tw/archives/81551,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,眼睛紅_20190821_3,2019-08-21,眼睛紅,常常眼睛紅紅、帶血絲？比起點眼藥水這2招更有效！ - Heho健康,雖然大家會說眼睛是我們的「靈魂之窗」，但卻很少人會每天盯著自己的眼睛觀察，總是要別人大叫「哇...,"[('眼睛', 30), ('眼藥水', 5), ('問題', 3), ('血管', 3),...","['雖然', '大家', '會', '說', '眼睛', '是', '我們', '的', '...","['眼睛', '靈魂', '眼睛', '眼睛', '白兔', '眼睛', '症狀', '動力...","[NerToken(word='0分', ner='TIME', idx=(787, 789...","[('雖然', 'Cbb'), ('大家', 'Nh'), ('會', 'D'), ('說'...",https://heho.com.tw/archives/55186,https://encrypted-tbn1.gstatic.com/images?q=tb...


### **再次過濾**

In [30]:
from collections import Counter

# 再次過濾: 兩個字以上、特定的詞性
# Filter condition: two words and specified POS
allowedPOS=['Na','Nb','Nc']
# 使用token_pos
def get_top_words():
        top_cate_words={} # final result
        counter_all = Counter() # counter for category '全部'
        for category in search_keys:
            
            # df yesterday-->This is your homework
            # df=df[(df.date >= start_date) & (df.date <= end_date)]
            
            # df category
            df_group = df[df.category == category]
            
            # concatenate all filtered words in the same category
            words_group = []
            for row in df_group.token_pos:
                
                # filter words for each news
                filtered_words =[]
                for (word, pos) in eval(row):
                    if (len(word) >= 2) & (pos in allowedPOS):
                        filtered_words.append(word)
                        
                # concatenate filtered words  
                words_group += filtered_words

            # now we can count word frequency
            counter = Counter( words_group )
            
            # counter 
            counter_all += counter
            topwords = counter.most_common(30)

            # store topwords
            top_cate_words[category]= topwords

        # Process category '全部'
        top_cate_words['全部'] = counter_all.most_common(30)

        return list(top_cate_words.items())

In [31]:
%%time
result = get_top_words()

result[0:2]

Wall time: 255 ms


[('眼睛紅',
  [('眼睛', 86),
   ('血管', 21),
   ('結膜', 19),
   ('角膜', 18),
   ('結膜炎', 13),
   ('角膜炎', 12),
   ('眼球', 9),
   ('視力', 9),
   ('問題', 8),
   ('症狀', 7),
   ('醫師', 6),
   ('疾病', 6),
   ('隱形眼鏡', 6),
   ('眼藥水', 6),
   ('時候', 6),
   ('眼科', 5),
   ('原因', 5),
   ('外傷', 5),
   ('結膜下出血', 5),
   ('細菌', 5),
   ('陳瑩山', 4),
   ('鏡子', 3),
   ('感覺', 3),
   ('情形', 3),
   ('主任', 3),
   ('組織', 3),
   ('血絲', 3),
   ('病毒', 3),
   ('毛巾', 3),
   ('眼藥', 3)]),
 ('呼吸道感染',
  [('呼吸道', 33),
   ('病毒', 27),
   ('症狀', 17),
   ('患者', 14),
   ('維生素', 13),
   ('疫苗', 13),
   ('維生素D', 12),
   ('風險', 12),
   ('兒童', 11),
   ('新冠肺炎', 9),
   ('孩童', 8),
   ('疾病', 7),
   ('功能', 7),
   ('活性', 7),
   ('喉嚨', 6),
   ('幼兒', 6),
   ('醫院', 6),
   ('家長', 6),
   ('嬰幼兒', 5),
   ('肺部', 5),
   ('來源', 5),
   ('併發症', 4),
   ('中醫', 4),
   ('感冒', 4),
   ('風寒', 4),
   ('黏膜', 4),
   ('免疫力', 4),
   ('黏液', 4),
   ('時期', 4),
   ('含量', 4)])]

In [32]:
row0 = df.iloc[0].token_pos
row0
# type(row0) 是 str

"[('小露', 'VC'), ('一早', 'D'), ('起床', 'VA'), ('照', 'VC'), ('鏡子', 'Na'), ('時', 'Ng'), ('發現', 'VE'), ('自己', 'Nh'), ('的', 'DE'), ('眼睛', 'Na'), ('紅紅', 'VH'), ('的', 'T'), ('，', 'COMMACATEGORY'), ('眼白', 'VH'), ('的', 'DE'), ('地方', 'Na'), ('像', 'P'), ('是', 'SHI'), ('在', 'P'), ('流血', 'VA'), ('一樣', 'VH'), ('，', 'COMMACATEGORY'), ('但', 'Cbb'), ('不', 'D'), ('痛', 'VH'), ('也', 'D'), ('不', 'D'), ('癢', 'VH'), ('，', 'COMMACATEGORY'), ('除了', 'P'), ('紅紅', 'VH'), ('的', 'DE'), ('看起來', 'D'), ('很', 'Dfa'), ('恐怖', 'VH'), ('，', 'COMMACATEGORY'), ('也', 'D'), ('沒有', 'VJ'), ('其他', 'Neqa'), ('不', 'D'), ('舒服', 'VH'), ('的', 'DE'), ('感覺', 'Na'), ('，', 'COMMACATEGORY'), ('眼科', 'Nc'), ('醫師', 'Na'), ('說', 'VE'), ('這', 'Nep'), ('是', 'SHI'), ('結膜', 'Na'), ('下', 'Ng'), ('出血', 'VH'), ('，', 'COMMACATEGORY'), ('主要', 'D'), ('是', 'SHI'), ('因為', 'Cbb'), ('太', 'Dfa'), ('過', 'Dfa'), ('勞累', 'VH'), ('，', 'COMMACATEGORY'), ('請', 'VF'), ('小露', 'VH'), ('要', 'D'), ('多', 'D'), ('休息', 'VA'), ('。', 'PERIODCATEGORY'), ('結膜', 'Na'), ('下', 'Ng'

In [33]:
# eval: 不論是整數或浮點數都會自動轉型成可以做計算的數值
row0 = eval(df.iloc[0].token_pos)
print("1.", len(row0))

row1 = eval(df.iloc[1].token_pos)
print("2.", len(row1))

row01 = row0+row1
print("3.", len(row01))

1. 557
2. 517
3. 1074


In [34]:
c0 =Counter( row0 )
c1 =Counter( row1 )

c0.most_common(10)
c1.most_common(10)

c01 = c0+c1
c01.most_common(10)

[(('，', 'COMMACATEGORY'), 80),
 (('的', 'DE'), 42),
 (('、', 'PAUSECATEGORY'), 28),
 (('。', 'PERIODCATEGORY'), 25),
 (('眼睛', 'Na'), 22),
 (('是', 'SHI'), 20),
 (('會', 'D'), 18),
 (('結膜', 'Na'), 15),
 (('：', 'COLONCATEGORY'), 14),
 (('就', 'D'), 14)]

In [35]:
top_group_words = get_top_words()
df_top_group_words = pd.DataFrame(top_group_words, columns = ['category','top_keys'])
df_top_group_words

,category,top_keys
0,眼睛紅,"[(眼睛, 86), (血管, 21), (結膜, 19), (角膜, 18), (結膜炎,..."
1,呼吸道感染,"[(呼吸道, 33), (病毒, 27), (症狀, 17), (患者, 14), (維生素..."
2,糖尿病,"[(糖尿病, 129), (身體, 36), (胰島素, 26), (血糖, 19), (症..."
3,耳鳴,"[(問題, 25), (症狀, 21), (原因, 17), (鼻咽癌, 17), (睡眠,..."
4,落髮,"[(頭髮, 58), (頭皮, 30), (女性, 18), (毛囊, 15), (醫師, ..."
5,憂鬱,"[(憂鬱症, 65), (情緒, 37), (醫師, 34), (症狀, 23), (患者,..."
6,全部,"[(糖尿病, 143), (症狀, 91), (眼睛, 88), (醫師, 67), (身體..."


In [36]:
df_top_group_words.to_csv('heho_topkey_with_category_via_token_pos.csv', index=False)

In [37]:
df_data = pd.read_csv('heho_topkey_with_category_via_token_pos.csv',sep=',')
# df_data.head(12)

# df_data['top_keys'][0] 為字串，並非list
eval(df_data['top_keys'][0])

[('眼睛', 86),
 ('血管', 21),
 ('結膜', 19),
 ('角膜', 18),
 ('結膜炎', 13),
 ('角膜炎', 12),
 ('眼球', 9),
 ('視力', 9),
 ('問題', 8),
 ('症狀', 7),
 ('醫師', 6),
 ('疾病', 6),
 ('隱形眼鏡', 6),
 ('眼藥水', 6),
 ('時候', 6),
 ('眼科', 5),
 ('原因', 5),
 ('外傷', 5),
 ('結膜下出血', 5),
 ('細菌', 5),
 ('陳瑩山', 4),
 ('鏡子', 3),
 ('感覺', 3),
 ('情形', 3),
 ('主任', 3),
 ('組織', 3),
 ('血絲', 3),
 ('病毒', 3),
 ('毛巾', 3),
 ('眼藥', 3)]

In [38]:
data={}
for idx, row in df_data.iterrows():
    data[row['category']]= eval(row['top_keys'])

data['眼睛紅'][0:6]
data['落髮'][0:10]
data['糖尿病'][0:6]

[('糖尿病', 129), ('身體', 36), ('胰島素', 26), ('血糖', 19), ('症狀', 18), ('原因', 17)]